In [1]:
from decaf.scripts.imagenet import DecafNet


We STRONGLY recommend you to try to install mpi and mpi4py.



Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
import os
f_dir = "/home/ce-ruoxu/workspace/data-fusion/"
os.chdir(f_dir)

In [3]:
wt_dir = 'data/decaf/imagenet_pretrained/'
%time net = DecafNet(wt_dir + 'imagenet.decafnet.epoch90',wt_dir + 'imagenet.decafnet.meta')

CPU times: user 755 ms, sys: 242 ms, total: 998 ms
Wall time: 1e+03 ms


In [59]:
import os
im_dir = "data/im2text/subset_4837/image/"
f_names = os.listdir(f_dir + im_dir)

In [26]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [50]:
# return 4096 d feature of the image
def feature_extractor(im_name):
    #layer = 'fc7_cudanet_out'
    import matplotlib.pyplot as plt
    layer = 'fc6_cudanet_out'
    result = plt.imread(im_name)
    scores = net.classify(result.astype('uint8'),center_only=True)
    feature = net.feature(layer)
    return feature.flatten()

In [28]:
from IPython import parallel
clients = parallel.Client()
clients.block = True
view = clients.load_balanced_view()

In [41]:
feature_extractor('672675')

array([-12.64466858, -18.77211189,  17.39270401, ..., -10.06385422,
        -0.81068993, -20.84035683], dtype=float32)

In [44]:
feature_extractor(f_names[1])

array([-17.55247116, -17.88421631, -16.76299858, ...,  -5.77874422,
       -10.85248661, -20.48731422], dtype=float32)

In [63]:
import time

0.00014199999999675583

In [70]:
### Extract image features
start = time.clock()
index = []
train_ims = []
for im_name in f_names:
    try:
        train_ims.append(feature_extractor(im_dir + im_name))
        index.append(im_name)
    except:
        pass
time.clock()-start
train_ims = np.array(train_ims)

In [75]:
import cPickle
with open("data/im2text/index_fc6_cudanet_out","wb") as f:
    cPickle.dump(index,f)
with open("data/im2text/features_fc6_cudanet_out","wb") as f:
    cPickle.dump(train_ims,f)